In [1]:
import pandas as pd
from src.student_eval.app.evaluation import models
df = pd.DataFrame({
    'user_id'   : ['alice','alice','bob','bob']*3,
    'skill_name': ['fractions']*6 + ['equations']*6,
    'correct'   : [0,1,0,0,  1,1,  1,0,0,1,1,1],
    'item_id'   : ['A','B','A','B']*3
})
df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1
print(df)

   user_id skill_name  correct item_id  order_id
0    alice  fractions        0       A         1
1    alice  fractions        1       B         2
2      bob  fractions        0       A         1
3      bob  fractions        0       B         2
4    alice  fractions        1       A         3
5    alice  fractions        1       B         4
6      bob  equations        1       A         1
7      bob  equations        0       B         2
8    alice  equations        0       A         1
9    alice  equations        1       B         2
10     bob  equations        1       A         3
11     bob  equations        1       B         4


In [2]:
user_id = df['user_id'].to_list()
skill_name = df['skill_name'].to_list()
correct = df['correct'].to_list()
item_id = df['item_id'].to_list()
order_id = df['order_id'].to_list()
subject_id = ["maths"]*len(user_id)

model=models.StudentModel()
model.csv_path = "data.csv"
model.model_path = "model.pkl"
results = model.update_dataset(order_id=order_id,
                               user_id=user_id,
                               skill_name=skill_name,
                               correct=correct,
                               item_id=item_id,
                               subject_id=subject_id)
print(results["students_states"])
print(results["skills_states"])

{'alice': {'learns': {'maths': {'fractions': 0.14652593139873318, 'equations': 0.18204125937060714}}, 'forgets': {'maths': {'fractions': 0.0, 'equations': 0.0}}}, 'bob': {'learns': {'maths': {'fractions': 0.019449874721090995, 'equations': 0.7396777464125578}}, 'forgets': {'maths': {'fractions': 0.0, 'equations': 0.0}}}}
{'maths': {'equations': {'prior': 0.0, 'learns': 0.6319302851430383, 'guesses': 0.43978393778233055, 'slips': 0.10386711708111339, 'forgets': 0.6411319366434435}, 'fractions': {'prior': 0.0, 'learns': 0.4881341320441835, 'guesses': 0.028452105443358067, 'slips': 0.08061901421240351, 'forgets': 0.006562922277734135}}}


In [3]:
order_id = [1, 2, 1]
skill_name = ["optics", "optics", "optics"]
user_id = ["carlos", "carlos", "alice"]
item_id = ["A", "B", "A"]        
        # Ponemos tema a cada asignatura
correct = [1, 0, 1]
subject_id = ["physics"]*len(user_id)
results = model.update_dataset(order_id=order_id,
                               user_id=user_id,
                               skill_name=skill_name,
                               correct=correct,
                               item_id=item_id,
                               subject_id=subject_id)
print(results["students_states"])
print(results["skills_states"])

{'alice': {'learns': {'maths': {'fractions': 0.14652593139873318, 'equations': 0.18204125937060714}, 'physics': {'optics': 0.03837908905107387}}, 'forgets': {'maths': {'fractions': 0.0, 'equations': 0.0}, 'physics': {'optics': 0.0}}}, 'bob': {'learns': {'maths': {'fractions': 0.019449874721090995, 'equations': 0.7396777464125578}}, 'forgets': {'maths': {'fractions': 0.0, 'equations': 0.0}}}, 'carlos': {'learns': {'physics': {'optics': 0.0304148235019519}}, 'forgets': {'physics': {'optics': 0.0}}}}
{'maths': {'equations': {'prior': 0.0, 'learns': 0.6319302851430383, 'guesses': 0.43978393778233055, 'slips': 0.10386711708111339, 'forgets': 0.6411319366434435}, 'fractions': {'prior': 0.0, 'learns': 0.4881341320441835, 'guesses': 0.028452105443358067, 'slips': 0.08061901421240351, 'forgets': 0.006562922277734135}}, 'physics': {'optics': {'prior': 0.0, 'learns': 0.01107315572712013, 'guesses': 0.6625575960752863, 'slips': 0.19279985022977425, 'forgets': 0.8127535932273806}}}


In [6]:
from pyBKT.models import Model, Roster

In [7]:
real_model = model.student_model
real_model.save("model2.pkl")
import pickle
with open("model2.pkl", "rb") as f:
    real_model_loaded = pickle.load(f)

In [11]:
roster = Roster(students=["alice", "bob"], skills =["fractions","optics"] , model=real_model)
roster_loaded = Roster(students=["alice", "bob"], skills =["fractions","optics"] , model=real_model_loaded)
print(roster.get_state("fractions", "alice"))
print(roster_loaded.get_state("fractions", "alice"))
print(roster.update_state("fractions", "alice", 1))
print(roster_loaded.update_state("fractions", "alice", 1))
state = roster.get_state("fractions", "alice")
print(roster_loaded.get_state("fractions", "alice"))
# Guardamos en pickle
import pickle
with open("roaster_model.pkl", "wb") as f:
    pickle.dump(roster, f)
# Cargamos el modelo
with open("roaster_model.pkl", "rb") as f:
    loaded_roster = pickle.load(f)
print(loaded_roster.get_state("optics", "alice"))
print(state.state_type.name)
print(state.current_state["state_prediction"])
print(state.current_state["correct_prediction"])


State(StateType.UNMASTERED, {'correct_prediction': 0.028452105443358067, 'state_prediction': 0.0}, Roster(...))
State(StateType.UNMASTERED, {'correct_prediction': 0.028452105443358067, 'state_prediction': 0.0}, Roster(...))
State(StateType.UNMASTERED, {'correct_prediction': 0.4633449011632891, 'state_prediction': 0.4881341320441835}, Roster(...))
State(StateType.UNMASTERED, {'correct_prediction': 0.4633449011632891, 'state_prediction': 0.4881341320441835}, Roster(...))
State(StateType.UNMASTERED, {'correct_prediction': 0.4633449011632891, 'state_prediction': 0.4881341320441835}, Roster(...))
State(StateType.UNMASTERED, {'correct_prediction': 0.6625575960752863, 'state_prediction': 0.0}, Roster(...))
UNMASTERED
0.4881341320441835
0.4633449011632891
